# Desarrollo
## Avances
- Cargadas las señales en formato MNE y preprocesadas, falta revisar si funciona correctamente

## Conclusiones
- Hay que ver como cargar el archivo de montage para MNE.

In [95]:
#LIBRERÍAS

#Librerías de Sistema
# import os
import sys

#Librerías de Manejo de Datos
import pandas as pd
import numpy as np
import scipy.signal as sgn

#Librerías de Interfaz
from tkinter import filedialog
from tkinter import *
from tkinter import messagebox

#Librerías para EEG
import mne

#Librerías para Plotings
import matplotlib.pyplot as plt

import plotly.graph_objs as go
import plotly.offline as py
import plotly

py.init_notebook_mode(connected = True) #Stablishing offline plot for Plotly

### Variables Globales 

In [60]:
#definimos las variables de antemano para que las use todo el algoritmo, como globales
usecols = 0   #columnas a utilizar del archivo de señal
skiprows = 0  #Filas a saltear del archivo de señal
sep = ","     #Símbolo que separa columnas en el archivo de señal

col_names = [] #Lista para nombre de las columnas
col_types = [] #Lista para el tipo de datos en cada columna

chan_quant = 0 #Cantidad de columnas que son del tipo "Channel"

chan_pos = []  #Posiciones en las que se colocaron los electrodos de cada canal
chan_selecc = [] #Lista de canales que se van a utilizar del total presentes en el archivo de señal

fq_lo = 0      #frequencia baja del filtro pasabanda
fq_hi = 0      #frecuencia alta del filtro pasabanda
fq_no = 0      #frecuencia del filtro notch
fq_sub = 0     #frecuencia de submuestreo

int_quant = 0
int_name= []
int_ini= []
int_end= []
int_ini_sample= []
int_end_sample= []


In [61]:
#Definir Layouts para Plotly
time_layout = go.Layout(title = "Will be changed in each plot",
                        xaxis = dict(title = 'Time [s]',
                                     rangeslider = dict(visible = True,
                                                        yaxis = dict(rangemode = 'auto'
                                                                    )
                                                        )
                                    ),
                        yaxis = dict(title = 'Amplitude [uV]',
                                     fixedrange = False)
                       )

freq_layout = go.Layout(title = "Will be changed in each plot",
                        xaxis = dict(title = 'Frequecy [Hz]',
                                     range = [0, fq_hi + 5],
                                     rangeslider = dict(visible = True,
                                                        yaxis = dict(rangemode = 'auto'
                                                                    )
                                                        )
                                    ),
                        yaxis = dict(title = 'Amplitude [uV]',
                                     fixedrange = False)
                       )

spec_layout = go.Layout(title = "Will be changed in each plot",
                        xaxis = dict(title = 'Time [s]'
                                    ),
                        yaxis = dict(title = 'Frequency [Hz]',
                                     range = [0, fq_hi+5],
                                     fixedrange = False)
                       )

### Funciones

In [62]:
#we create the function get indexes
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]

#This would work like : X = ["Mario", "Luigi", "Yoshi", "Mario"]
#print(get_indexes("Mario", X)) -> Out: [0,3]
#print(get_indexes("Luigi", X)) -> Out: [1]

In [63]:

def yn_quest (quest):
    yn_quest_win = Tk()
    
    answer = messagebox.askquestion("QUEST",quest)
    yn_quest_win.destroy()
    
    yn_quest_win.mainloop()
    
    return answer


In [64]:
def load_signal():   
    nom_s = input("Wich will be the signal name? ")

    while True:
        search_signal_win = Tk() #Crea la ventana root
        xs = filedialog.askopenfilename(title = "Select Signal File :",          #Dice que va a sacar la variable Xs como el filename de un archivo
                                        filetypes = (("Documento de texto","*.txt"),
                                                     ("Excel (post 2003)", ".xlsx"),
                                                     ("Excel (prev 2003)", ".xls"),
                                                     ("Todos los archivos","*.*")
                                                    ) #Con titulo de ventana y opciones de abrir archivos de ese tipo
                                       )

        search_signal_win.destroy()
        search_signal_win.mainloop() #esto es para que la ventana quede abierta hasta que se de una respuesta, en este caso la selección de un archivo

        print (xs)

        if ".txt" in xs:
            opt = yn_quest('Is it a file exported by the OpenBCI Software?')
            if opt == 'Yes' or opt == 'yes':
                XS = openbci_topd(xs)
            else:
                XS = csv_topd(xs)
            print("Signal Loaded. TXT format.")
            break
        
        elif ".xlsx" in xs or ".xls" in xs:
            XS = excel_topd(xs, interval = False)
            print("Signal Loaded. Excel format.") #Esto sería reemplazado por la función "leer txt" que tomaría la dirección y parámetros como Cantidad de columnas, Nombres de columnas, etc (o se puede probar con la direccion sola y si no funciona pedir parametros)
            break

        elif xs == "":
            print("Cancelled Operation")
            break

        else:
            print("Wrong File Format. Try Again.")
            break
    
    return XS, nom_s

In [65]:
def csv_topd(truepath):
    global usecols
    global skiprows
    global sep
    global col_names
    global col_types
    
    while True:
        col_rows_sep()
        
        save_opt = input('Do you wish to save this configuration?: ') #Completar, cómo y dónde y la carga de la configuracion
        
        try:
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, names=col_names, usecols=range(usecols), lineterminator = '\n')
        except:
            print('\n\nThe input parameters do not match. Please enter them again.\n\n')
            
        else:
            break
    
    X.col_names = col_names #Esto sería medio de gusto si ahora col_names y col_types son globales
    X.col_types = col_types
    
    return X

In [66]:
def openbci_topd(truepath):
    global usecols
    global skiprows
    global sep
    global col_names
    global col_types
    
    selecc_predet = yn_quest('Will you use the default OpenBCI configuration?')
    
    if selecc_predet == 'No' or selecc_predet == 'no': #si decide que no el usuario ingresa los datos a tener en cuenta al leer, pero un archivo txt de OpenBCI
        col_rows_sep()
        
        save_opt = input('Do you wish to save this configuration?: ') #Completar, cómo y dónde y la carga de la configuracion
        
    elif selecc_predet == 'Yes' or selecc_predet == 'yes':
        sep = ","
        col_names = ['Sample', 'Ch1', 'Ch2', 'Ch3', 'Ch4', 'x', 'y', 'z', 'Time', ' ']
        col_types = ['Misc', 'Channel', 'Channel', 'Channel', 'Channel', 'Accel', 'Accel', 'Accel', 'Time', 'Misc'] #Esto sirve para saber que cantidad de columnas son canales, y que canal tiene los marcadores
        usecols = 10
        skiprows = 6
        
    while True:        
        try:           
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')

            assert X.values[0,0] == 1 

        except AssertionError:   
            skiprows = int(input("The selected format does not fit the file selected. ¿How many rows must be skiped?: "))

            Xt = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')
            if all(X.values[0:4,0] != [0,1,2,3]):
                sep = ";"
                Xt = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')

            if all(X.values[0:4,0] != [0,1,2,3]):
                sep = "."
                Xt = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, lineterminator = '\n')

            if all(X.values[0:4,0] != [0,1,2,3]):
                sep = input("The selected format does not fit the file selected. Enter how columns are separated in the CSV file (.txt): ")
#Faltaría un contador que vuelva hacia atrás si aún ingresando el formato manualmente sigue sin encajar,porque talvez no tiene la columna samples, podria aprovechar el return
        else:
            break

    #vamos a hacer dos whiles infinitos, para que en el primero no se preocupe por los nombres y cantidad de columnas, sino por cuantas filas saltear, y en este próximo vea columnas

    while True:        
        try:           
            X = pd.read_csv(truepath,  sep=sep, skiprows=skiprows, names=col_names, usecols=range(usecols), lineterminator = '\n')
        except:                   
            usecols = int(input("The selected format does not fit the file selected. Please enter the number of columns: "))
            col_names = []
            col_types = []
            for i in range(usecols):
                print("Name of column number ", (i+1))
                nombre = input("Enter: ")
                print("Type of column number ", (i+1), " [Channel, Time, Accel, Markers, Misc]") #Si no se puede armar la seleccion con Tkinter, buscar la librería pick de wong2
                tipo = input("Enter: ")
                col_names.append(nombre)
                col_types.append(tipo)
        else:                     
            break 
    
    return X

In [67]:
def excel_topd(truepath, interval = True):
    global usecols
    global skiprows
    global col_names
    global col_types
    
    while True:
        if interval == True:
            
            header = 0
            index_col = 0
            usecols = 3
            skiprows = 0
            col_names = ['Name', 'Startpoint']

            endpoint_opt = yn_quest("Does it has an endpoint column?")
            if endpoint_opt == 'Yes' or endpoint_opt == 'yes':
                col_names.append('Endpoint')
                usecols =+ 1

            else:
                duration_opt = yn_quest("Does it has a duration column?")
                if duration_opt == 'Yes' or duration_opt == 'yes':
                    col_names.append('Duration')
                    usecols =+ 1

        elif interval == False:
            header = None
            index_col = None
            col_rows_sep()
        
        try:
            X = pd.read_excel(truepath, usecols = range(usecols), skiprows = skiprows, names = col_names, header=header, index_col=index_col)
        except:
            print('\n\nThe input parameters do not match. Please enter them again.\n\n')
        else:
            break
    
    return X

In [68]:
def col_rows_sep():
    
    def accept_col():
        col_names_win.destroy()
        global col_names
        global col_types
        
        col_names = []
        col_types = []
        for i in range(columnas.get()):
            col_names.append(col_n[i].get())
            col_types.append(col_t[i].get())
                    
    def selec_pd ():       
        
        conf = messagebox.askquestion("Confirm","Will you select : " + str(columnas.get()) + " columns & " + str(filas.get()) + " rows?")
        if conf == "yes":
            global usecols   # cada vez que las queremos modificar tenemos que decirle que nos referimos a las globales
            global skiprows  #Deberíamos buscar una solución para esto, ya que no es muy bueno, pero no podemos retornar
            global sep        #correctametne las variables porque estan en una función anidada
            usecols = columnas.get()   
            skiprows = filas.get()     
            sep = separador.get()
                        
            selec_pd_win.destroy()
            global col_names_win 
            col_names_win = Tk()
            col_names_win.title("Columns")
            col_names_win.label1 = Label(col_names_win, text = "Name:").place(x=30,y=20)
            col_names_win.label2 = Label(col_names_win, text = "Type:").place(x=170,y=20)
        
            col_names_win.geometry("350x"+str(40+columnas.get()*35))
            for i in range(columnas.get()):
                col_n.append(StringVar())
                col_t.append(StringVar())
                col_names_win.entry = Entry(col_names_win, textvariable=col_n[i]).place(x=30 , y=40+i*26)
                col_names_win.spinbox = Spinbox(col_names_win, values = ("Channel", "Misc", "Tag", "Time", "Accel"), wrap =True, textvariable =col_t[i]).place(x=170 , y=40+i*26)
            col_names_win.button1 = Button (col_names_win , text="Accept", command=accept_col).place(x=100, y=40+26*columnas.get()) #command= imprimir)
            col_names_win.mainloop()

    selec_pd_win = Tk() #Crea la ventana
    
    col_n = []
    col_t = []
    
    columnas = IntVar()
    filas = IntVar()
    separador = StringVar()

    selec_pd_win.title("Set parameters for Pandas" ) 
    selec_pd_win.geometry("350x200")

    selec_pd_win.label1 = Label(selec_pd_win , text= "Rows to Skip:").place(x=30,y=20)
    selec_pd_win.entry1 = Entry(selec_pd_win, textvariable=filas).place(x=170 , y=20)

    selec_pd_win.label2 = Label(selec_pd_win, text="Columns to use:").place(x=30 , y=60)
    selec_pd_win.entry2 = Entry(selec_pd_win, textvariable=columnas).place(x=170 , y=60)

    selec_pd_win.label3 = Label(selec_pd_win, text="Separator to use:").place(x=30 , y=100)
    selec_pd_win.entry3 = Entry(selec_pd_win, textvariable=separador).place(x=170 , y=100)

    selec_pd_win.button1 = Button (selec_pd_win , text="Accept", command=selec_pd).place(x=170 , y=150) #command= imprimir)

    selec_pd_win.mainloop()

In [69]:
def chan_config(col_names, col_types):
    
    def selec_chan():
        global chan_pos
        global chan_selecc
        chan_pos = []
        chan_selecc = []
        
        chan_selec_win.destroy()
        for i in range(chan_quant):
            chan_pos.append(chan_p[i].get())
            chan_selecc.append(chan_s[i].get())
    
    global chan_quant
    chan_quant = 0
    chan_all = []
    chan_p = []
    chan_s = []
    
    i = 0
    for word in col_types:
        if word == 'Channel':
            chan_all.append(col_names[i])
            chan_quant += 1
        i += 1

    chan_selec_win = Tk()
    chan_selec_win.title("Channel Selection")
    chan_selec_win.label1 = Label(chan_selec_win, text = str(chan_quant) +' Channels were detected, wich ones will you use?').place(x=30,y=10)

    chan_selec_win.geometry("500x"+str(80+chan_quant*35))
    for i in range(chan_quant):
        chan_s.append(IntVar())
        chan_p.append(StringVar())

        chan_selec_win.label2 = Label(chan_selec_win, text = chan_all[i] + " column:").place(x=20 , y=40+i*26)
        chan_selec_win.check1 = Checkbutton(chan_selec_win, text = "Import?", variable=chan_s[i], onvalue=1, offvalue=0).place(x=90 , y=40+i*26)
        chan_selec_win.label3 = Label(chan_selec_win, text = "Channel Position:").place(x=180 , y=40+i*26)
        chan_selec_win.entry1 = Entry(chan_selec_win, textvariable = chan_p[i]).place(x=300 , y=40+i*26)

    chan_selec_win.button1 = Button (chan_selec_win , text="Accept Channels", command=selec_chan).place(x=100, y=60+26*chan_quant) #command= imprimir)

    chan_selec_win.mainloop()

In [70]:
def pd_toarray(X_pd):
    
    i = 0
    j = 0
    chan_list = []
    accel_list = []
    tag_list = []
    global col_types
    global chan_selecc
    
    while i < X_pd.shape[-1]:
        if col_types[i] == "Channel":
            if chan_selecc[j] == 1:
                chan_list.append(X_pd.columns[i])
            j += 1

        if col_types[i] == " Accel":
            accel_list.append(X_pd.columns[i])

        if col_types[i] == "Tag":
            tag_list.append(X_pd.columns[i])
        i += 1

    X_chan = (X_pd.filter(chan_list)).values
    X_accel = (X_pd.filter(accel_list)).values
    X_tag = (X_pd.filter(tag_list)).values
    
    return X_chan, X_accel, X_tag

In [71]:
def array_tomneRaw(X_chan):
    global chan_selecc
    global chan_pos
    ch_names = []
    #montage = []

    for i in get_indexes(1,chan_selecc):
        ch_names.append('Ch'+ str(i+1))
        #montage.append(chan_pos[i])   No funciona así, porque hay que darle una variable montage especifica, hay que ver como crearla

    X_mneinfo = mne.create_info(ch_names = ch_names, 
                                ch_types = 'eeg', 
                                #montage = montage,  ver como crear el montage https://martinos.org/mne/dev/generated/mne.channels.Montage.html#mne.channels.Montage
                                sfreq = 200) #Cambiar por fq_sub

    X_mneRaw = mne.io.RawArray(np.transpose(X_chan), info = X_mneinfo)
    
    return X_mneRaw #ver si también retornamos X_mneinfo


In [72]:
def interval_choice ():  #Request the quantity and size of de intervals
    
    global int_quant
    global int_name
    global int_ini
    global int_end
    global int_ini_sample
    global int_end_sample
    global fq_sub
    
    int_quant = 0
    int_name= []
    int_ini= []
    int_end= []
    int_ini_sample= []
    int_end_sample= []

    
    Excel_int= yn_quest("Do you wish to load an excel file with intervals?")
    
    if Excel_int== "Yes" or Excel_int=="yes":  
        
        search_int_win = Tk() #Crea la ventana root
        int_path = filedialog.askopenfilename(title = "Select Signal File :",          #Dice que va a sacar la variable Xs como el filename de un archivo
                                        filetypes = (("Excel (post 2003)", ".xlsx"),
                                                     ("Excel (prev 2003)", ".xls"),
                                                     ("All files","*.*")
                                                    ) #Con titulo de ventana y opciones de abrir archivos de ese tipo
                                       ) 
        search_int_win.destroy()
        
        int_pd = excel_topd(int_path)
        
        int_quant = int_pd.shape[0]
        
        for i in range(int_quant):
        
            int_name.append(int_pd.values[i,0])
            int_ini.append(int_pd.values[i,1])
            
            if 'Endpoint' in int_pd.columns:
                int_end.append(int_pd.values[i,2])
            
            elif 'Duration' in int_pd.columns:
                int_end.append(int_pd.values[i,1] + int_pd.values[i,2])
            
            else:
                if i == 0:
                    int_durat = int(input("How long did the intervals last (in sec)?: "))
                int_end.append(int_pd.values[i,1] + int_durat)
                
            sample_ini= (int_ini[i]*int(fq_sub))
            sample_end= (int_end[i]*int(fq_sub))

            int_ini_sample.append(sample_ini)
            int_end_sample.append(sample_end)
        
        search_int_win.mainloop()
        
    if Excel_int== "No" or Excel_int=="no":
        int_quant = int(input("How many interval will you analyze?: "))
        i = 0

        #Stablish names and sizes
        while i < int_quant:
            print("Name for the interval no. ", (i+1))
            int_name.append(input())
            #Solicito al usuario los rangos de tiempo para cada intervalo
            print("Start time for interval " + int_name [i] , end ="\n")
            int_ini.append(int(input('Enter in secs: '))) 
            print("End time for interval " + int_name [i] , end ="\n")
            int_end.append(int(input('Enter in secs: '))) 
            #Multiplico esos tiempos por la frecuencia de submuestreo, para obtener la cantidad de muestras que quiero ver
            sample_ini= (int_ini[i]*int(fq_sub))
            sample_end= (int_end[i]*int(fq_sub))

            int_ini_sample.append(sample_ini)
            int_end_sample.append(sample_end)
            i = i + 1
            
    print(int_name)
    print(int_ini)
    print(int_end)
    print(int_ini_sample)
    print(int_end_sample)
    print(fq_sub)

In [73]:
def sgn_preproc(X_mneRaw):
    global fq_lo
    global fq_hi
    global fq_no
    global fq_sub
    
    filt_order = input("Insert Butterworth filter Order (if blank, default is 5): ")
    
    if filt_order == '':
        filt_order = 5
    else:
        filt_order = int(filt_order)
        
    fq_lo = input("Insert Low Frequency for Bandpass Filter (leave blank space for no filtering): ")
    
    if fq_lo == '':
        fq_lo = None
    else:
        fq_lo = float(fq_lo)
    
    fq_hi = input("Insert High Frequency for Bandpass Filter (leave blank space for no filtering): ")
    
    if fq_hi == '':
        fq_hi = None
    else:
        fq_hi = float(fq_hi)
    
    filtered_signal = X_mneRaw.copy()
    
    filtered_signal.filter(l_freq = fq_lo,
                           h_freq = fq_hi,
                           method = 'iir',
                           iir_params = dict(order = filt_order,
                                             ftype = "butter"
                                            )
                        )
    
    fq_no = input("Insert Frequency for Notch Filter (leave blank space for no filtering): ") #Se puede agregar para que filtre en varias frecuencias con notch, pasando una lista de floats
    
    if fq_no == '':
        fq_no = None
    else:
        fq_no = float(fq_no)
        
    filtered_signal.notch_filter(freqs = fq_no,
                                notch_widths = 1,
                                method = 'iir',
                                iir_params = dict(order = filt_order,
                                                  ftype = "butter"
                                                 )
                                )
    
    fq_sub = input("Insert Subsample Frequency (Insert 200 for no Subsampling): ")
    
    if fq_sub != '' and fq_sub != '200':
        fq_sub = float(fq_sub)
        filtered_signal.resample(sfreq = fq_sub)
        return filtered_signal
    else:
        fq_sub=200
        return filtered_signal

In [93]:
def plot_time ():
    
    selecc_grafRaw = yn_quest('Do you wish to plot the Raw Signal?')
    selecc_grafProc = yn_quest('Do you wish to plot the Proc Signal?')
    
    for i in range(int_quant):  #Iria en vez de int_quant algún int_selecc que indique solo intervalos seleccionados
        raw_scat = []
        proc_scat = []
 
        for j in range(X_mneRaw.get_data().shape[0]):
            raw_scat.append(go.Scatter(x = np.arange(int_ini[i], int_end[i], 1/fq_sub),
                                       y = X_mneRaw.get_data()[j, int_ini_sample[i]:int_end_sample[i]],
                                       name = X_mneRaw.ch_names[j] #cambiar despues por posicion en montage, cuando montage este implementado
                                      )
                           )
            proc_scat.append(go.Scatter(x = np.arange(int_ini[i], int_end[i], 1/fq_sub),
                                        y = X_mneProc.get_data()[j, int_ini_sample[i]:int_end_sample[i]],
                                        name = X_mneProc.ch_names[j] #cambiar despues por posicion en montage, cuando montage este implementado
                                       )
                           )
            
            
        time_layout.title = int_name[i]
        
        if selecc_grafRaw == 'Yes' or selecc_grafRaw == 'yes': 
            raw_fig = go.Figure(data = raw_scat, layout = time_layout)
            
            py.iplot(raw_fig)
            
        if selecc_grafProc == 'Yes' or selecc_grafProc == 'yes':
            proc_fig = go.Figure(data = proc_scat, layout = time_layout)
    
            py.iplot(proc_fig)
        

In [129]:
def sgn_fft(signal):
    
    fft_list = [] #this llist will contain one list for each interval, with the content of each_fft_list
    
    welch_opt = yn_quest("Will you use Welch's method for tranforming?")
    
    if welch_opt == 'Yes' or welch_opt == 'yes':
        welch_twin = int(input('How long shall the transform window be (in secs)?: '))
        welch_twin_overlap = int(input('How many seconds will be overlapped?: '))
        
        for i in range(int_quant):
            
            each_fft_list = [] #on each iteration for the intervals this list will contain the freq bins and transform for each channel
            
            for j in range(X_mneRaw.get_data().shape[0]):
                
                fr, tf = sgn.welch(signal[j, int_ini_sample[i]:int_end_sample[i]], 
                                   fs=fq_sub, 
                                   nperseg = fq_sub*welch_twin, 
                                   noverlap = fq_sub*welch_twin_overlap)
                if j == 0:
                    each_fft_list.append(fr)
                
                each_fft_list.append(tf)
                
            fft_list.append(each_fft_list)
    
    else:
        
        for i in range(int_quant):
            
            each_fft_list = [] #on each iteration for the intervals this list will contain the freq bins and transform for each channel
            
            fr = np.fft.fftfreq(int_end[i]-int_ini[i], d = 1/fq_sub)
            each_fft_list.append(fr)
            
            for j in range(X_mneRaw.get_data().shape[0]):
                
                tf = np.fft.fft(signal[j, int_ini_sample[i]:int_end_sample[i]], norm = 'ortho')
                each_fft_list.append(tf)
            
            fft_list.append(each_fft_list)
                                
    
    return(fft_list)       

### Cuerpo del Código

In [74]:
X_pd, signalname = load_signal()

chan_config(col_names, col_types)

X_chan, X_accel, X_tag = pd_toarray(X_pd)

X_mneRaw = array_tomneRaw(X_chan)

X_mneProc = sgn_preproc(X_mneRaw)

interval_choice() 

plot_time()

FFT_matrix = sgn_fft(X_mneProc.get_data())

Wich will be the signal name? agus
C:/Users/toshiba pc/Downloads/96cb-pruebabarridofrec-AR-26-11-18.txt
Do you wish to save this configuration?: N
Signal Loaded. TXT format.
Creating RawArray with float64 data, n_channels=2, n_times=172851
    Range : 0 ... 172850 =      0.000 ...   864.250 secs
Ready.
Insert Butterworth filter Order (if blank, default is 5): 5
Insert Low Frequency for Bandpass Filter (leave blank space for no filtering): 5
Insert High Frequency for Bandpass Filter (leave blank space for no filtering): 60
Setting up band-pass filter from 5 - 60 Hz
Insert Frequency for Notch Filter (leave blank space for no filtering): 50
Setting up band-stop filter from 49 - 51 Hz
Insert Subsample Frequency (Insert 200 for no Subsampling): 200
How many interval will you analyze?: 3
Name for the interval no.  1
jede
Start time for interval jede
Enter in secs: 5
End time for interval jede
Enter in secs: 150
Name for the interval no.  2
troto
Start time for interval troto
Enter in secs: 1

In [137]:
lista1 = ['yo', 'soy', 'jm']
lista2= ['yo', 'soy', lista1]
lista3 = [lista1, lista2]

lista3[1][2][2]

'jm'

In [142]:
FFT_matrix = sgn_fft(X_mneProc.get_data())

In [144]:
FFT_matrix[j][0]

NameError: name 'j' is not defined

In [147]:
try:
    1/0
except:
    print('mal')

mal
